## Solution: Regression minimizing $\ell_1$-error

First of all, let us paste the observed values of age and height into this notebook.

In [ ]:
# Observed ages in years
obs_age = [1,1,2,2,3,4,4,5,6,7,7,8,9,9,10]

# Observed heights in cm
obs_ht = [73.2,73.3,75.1,77.4,80.1,85.7,84.0,89.1,100.2,105.3,112.2,115.0,125.1,135.2,150.7]

---

### Finding the optimal line: How to write an LP?

As stated in the problem, our goal is to find parameters $\alpha$ and $\beta$ such that the line $H = \beta A + \alpha$ minimizes the $\ell_1$-error

$$
\sum_{i=1}^{15} |\text{obs_ht}_{~i} - (\beta \times \text{obs_age}_{~i} + \alpha)|.
$$

Formulated differently, we want to solve

$$
\min_{\alpha, \beta\in\mathbb{R}} \sum_{i=1}^{15} |\text{obs_ht}_{~i} - (\beta \times \text{obs_age}_{~i} + \alpha)|
$$

Note that the objective is a sum of absolute values, which cause a problem (from a linear programming point of vies), as the absolute value function is not linear. Let us set $e_i = |\text{obs_ht}_{~i} - (\beta \times \text{obs_age}_{~i} + \alpha)|$ and rewrite the above as

$$
\begin{array}{rrcll}
\min & \sum_{i=1}^{15} e_i\\
 & e_i & = & |\text{obs_ht}_{~i} - (\beta \times \text{obs_age}_{~i} + \alpha)| & \forall i\in \{1,\ldots,15\}\\
 & \alpha, \beta & \in & \mathbb{R}.
\end{array}
$$

Now the objective looks linear, but there are constraints of the form $e_i = |\text{something linear in }\alpha\text{ and }\beta|$. These are obviously non-linear.

#### Modelling absolute values

To simplyfy things, think of a constraint of the form $e_i=|f|$ for two real-valued variables $e_i$ and $f$. How can we turn this into linear constraints?

Well, first of all, it is easy to see that

$$ e_i \geq |f| \quad\iff\quad \left\{\begin{array}{rc} & e_i \geq f \\ \text{and} & e_i \geq -f \end{array}\right.\quad. $$

From this, we can also get

$$ e_i = |f| \quad\iff\quad e_i\text{ is minimal such that } e_i \geq f \text{ and } e_i \geq -f\ . $$

This looks promising, so can we just rewrite the above optimization problem as

$$
\begin{array}{rrcll}
\min & \sum_{i=1}^{15} e_i\\
 & e_i & \geq & \text{obs_ht}_{~i} - (\beta \times \text{obs_age}_{~i} + \alpha) & \forall i\in \{1,\ldots,15\}\\
 & e_i & \geq & -\left(\text{obs_ht}_{~i} - (\beta \times \text{obs_age}_{~i} + \alpha)\right) & \forall i\in \{1,\ldots,15\}\\
 & \alpha, \beta & \in & \mathbb{R},
\end{array}
$$

and then solve this linear (!) program and that's it?

The answer turns out to be yes, but let's proof this slowly. Consider an optimal solution of the above linear program. We will show that in this case, we indeed have $e_i = |\text{obs_ht}_{~i} - (\beta \times \text{obs_age}_{~i} + \alpha)|$ for all $i\in \{1,\ldots,15\}$, as desired. By the constraints, we have $e_i \geq |\text{obs_ht}_{~i} - (\beta \times \text{obs_age}_{~i} + \alpha)|$. If for some $i$, we would actually have $e_i > |\text{obs_ht}_{~i} - (\beta \times \text{obs_age}_{~i} + \alpha)|$, then we could reduce the value of this $e_i$, which would improve the objective, thus contradicting optimality.

Thus, an optimal solution of the linear program corresponds to an optimal solution of the initial problem.

---

### Implementing and solving the LP

We now implement and solve the above linear program in pulp.

In [ ]:
# let n denote the number of observations
n = len(obs_age)
# assert that we have the same number of age and height observations
assert(n == len(obs_ht))

In [ ]:
# load pulp and set up the lp
import pulp
linRegLP = pulp.LpProblem("Linear regression", pulp.LpMinimize)

# create variables
alpha = pulp.LpVariable("alpha")
beta = pulp.LpVariable("beta")
e = [pulp.LpVariable(f"e_{i}") for i in range(n)]

In [ ]:
# set objective and check it
linRegLP.setObjective(pulp.lpSum(e))
linRegLP.objective

In [ ]:
# add constraints and check them
for i in range(n):
    rhs = obs_ht[i] - (beta * obs_age[i] + alpha)
    linRegLP.addConstraint(e[i] >= rhs, f"e_{i}_pos")
    linRegLP.addConstraint(e[i] >= -rhs, f"e_{i}_neg")
linRegLP.constraints

In [ ]:
# solve
linRegLP.solve()

print(f"The linear fuction interpolating optimally is\n" 
      + f"  H = {beta.value()} * A + {alpha.value()}")

In [ ]:
# plot
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# Plot the points
plt.scatter(obs_age,obs_ht)

# Draw the optimal line
A = np.linspace(1,10,100)
H = beta.value()*A+alpha.value()
plt.plot(A, H, color = 'red')

plt.xlabel("Age (in years)")
plt.ylabel("Height (in cm)")
plt.show()

---

### Finding the optimal quadratic function

We proceed similarly to the above, which gives the linear program

$$
\begin{array}{rrcll}
\min & \sum_{i=1}^{15} e_i\\
 & e_i & \geq & \text{obs_ht}_{~i} - (\alpha_2 \times \text{obs_age}_{~i}^2 + \alpha_1 \times \text{obs_age}_{~i} + \alpha_0) & \forall i\in \{1,\ldots,15\}\\
 & e_i & \geq & -\left(\text{obs_ht}_{~i} - (\alpha_2 \times \text{obs_age}_{~i}^2 + \alpha_1 \times \text{obs_age}_{~i} + \alpha_0)\right) & \forall i\in \{1,\ldots,15\}\\
 & \alpha_i & \in & \mathbb{R} & \forall i\in \{0,1,2\},
\end{array}
$$

As before, we can prove that an optimal solution to the above LP is indeed an optimal solution to the regression problem.

Implementing and solving this LP in pulp and plotting the resulting curve, we get the following.

In [ ]:
# set up the lp
quadRegLP = pulp.LpProblem("Quadratic regression", pulp.LpMinimize)

# create variables
alpha = [pulp.LpVariable(f"alpha_{i}") for i in range(3)]
e = [pulp.LpVariable(f"e_{i}") for i in range(n)]

# set objective
quadRegLP.setObjective(pulp.lpSum(e))

# add constraints and check them
for i in range(n):
    rhs = obs_ht[i] - (alpha[2] * obs_age[i]**2 + alpha[1] * obs_age[i] + alpha[0])
    quadRegLP.addConstraint(e[i] >= rhs, f"e_{i}_pos")
    quadRegLP.addConstraint(e[i] >= -rhs, f"e_{i}_neg")

# solve
quadRegLP.solve()

print(f"The quadratic fuction interpolating optimally is\n" 
      + f"  H = {alpha[2].value()} * A^2 + {alpha[1].value()} * A + {alpha[0].value()}")

In [ ]:
# plot
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# Plot the points
plt.scatter(obs_age,obs_ht)

# Draw the optimal quadratic function
A = np.linspace(1,10,100)
H = alpha[2].value()*A**2 + alpha[1].value()*A + alpha[0].value()
plt.plot(A, H, color = 'red')

plt.xlabel("Age (in years)")
plt.ylabel("Height (in cm)")
plt.show()